In [1]:
import visa                             # Import PyVISA library
import time
#import VISAresourceExtentions
#import numpy as np

In [2]:
# Initialization

rm = visa.ResourceManager()             # Create Resource Manager object
rs = rm.list_resources()                # Method to list the available resources
print(rs[0])                            
counter = rm.open_resource(rs[0])       # Assign the returned object to the instrument variable (i.e. counter)

USB0::5355::145::457158::0::INSTR


In [3]:
# Initial settings

print(counter.query('*IDN?'))           # Query the Identification string 
counter.write('*RST;*CLS')              # Reset the instrument, clear the Error queue
counter.timeout = 1e7                   # Acquisition timeout (ms) - set it higher than the acquisition time
counter.query('SYST:ERR?')              # Error Checking 

PENDULUM, CNT-91, 457158, V1.31 06 Mar 2015 12:55



'0,"No error"\n-'

In [4]:
#Basic settings

counter.write('CONF:FREQ (@1)')                       # Setup for frequency measurement from channel A (@1) 
counter.write('CALC:AVER:STAT OFF')                   # Enable/disable statitics
counter.write('INP:LEV:AUTO OFF; :INP:LEV 0')         # Enable/Disable autotrigger on channel A; Trigger level (V)
counter.write('FORMAT:TINF ON; :FORMAT ASCII')        # Read timestamp of each measurement (can also be omitted with CONF and MEAS); Redout in ASCII/REAL mode -> [freq (Hz), tstamp (sec)]
counter.write('SENSE:ACQ:APER 2e-8')                  # Gate time (sec), minimum: 20ns
counter.query('SYST:ERR?')                            # Error Checking                                                     

'0,"No error"\n-'

In [5]:
# Perform and read N consecutive individual measurements (Dead time = 10ms)

start = time.time()                                                        
                                                    
for i in range(100): 
    
    #counter.write('READ?')                                                 
    #counter.read()                                                         # write & read == query
    #freqtime = counter.query('READ?')                                      # Fetch list with string characters as its elements
    #freqtime = counter.query_ascii_values('READ?')                         # Fetch list of ASCII values ( ValueError: 'could not convert string to float can occur' )
    freqtime = counter.query_ascii_values('READ?', converter = 's')        # Fetch and Convert list of ASCII values into a list of strings
    
    freqtime[-1] = freqtime[-1].split('\n')[0]                             # Delete in the last element of the list of strings every character after the number ( i.e. \n0E..)
    freqtime = [float(i) for i in freqtime]                                # Convert list of strings into float numbers
    
    print(freqtime)
    
end = time.time()
print(end-start)                                                           # Total acquisition time (sec)
counter.query('SYST:ERR?')                                                 # Error Checking

[38900000.0, 29.79926113866]
[38890000.0, 29.82598863928]
[38900000.0, 29.836916032]
[38900000.0, 29.84778554769]
[38860000.0, 29.85871845012]
[38850000.0, 29.86964759423]
[38880000.0, 29.88057400833]
[38860000.0, 29.89138394232]
[38870000.0, 29.90213923685]
[38910000.0, 29.91295911268]
[38870000.0, 29.92380303016]
[38860000.0, 29.93472378432]
[38880000.0, 29.94563006098]
[38820000.0, 29.95658196668]
[38850000.0, 29.96743829634]
[38880000.0, 29.97823962914]
[38880000.0, 29.98929158996]
[38850000.0, 30.01625261397]
[38880000.0, 30.02727969975]
[38870000.0, 30.03823448773]
[38880000.0, 30.04918005241]
[38870000.0, 30.05986814898]
[38860000.0, 30.07070748385]
[38850000.0, 30.08149599183]
[38870000.0, 30.09226148406]
[38890000.0, 30.10322734039]
[38860000.0, 30.11415308613]
[38870000.0, 30.12531848635]
[38860000.0, 30.13615915742]
[38880000.0, 30.14716363898]
[38880000.0, 30.1579634746]
[38870000.0, 30.16882913009]
[38920000.0, 30.17974910123]
[38880000.0, 30.190709659]
[38890000.0, 30.217

'0,"No error"\n.'

In [6]:
counter.write('*RST;*CLS') 

(24, <StatusCode.success: 0>)

In [7]:
counter.close()